# Information Retrieval in High Dimensional Data
## Lab 8

|     |     |
| --- | --- |
| **Name:** | Uzair Akbar |
| **Matriculation Number:** | 03697290 |
| **E-mail:** | [uzair.akbar@tum.de](mailto:uzair.akbar@tum.de) |

## SVM
### Task 1
In this task, we will once again work with the MNIST training set. Prepare a training set matrix `X_train` consisting of the first 500 vectorized training samples of digits 1 and 2 each, and a corresponding label vector `y_train`. Use 1 and -1 for the labels.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from imageio import imread
from cvxopt import matrix, solvers

N=500
digit_prefix=['d1','d2']
X_train=np.zeros((784,2*N))
for i,dp in enumerate(digit_prefix):
    for j in range(N):
        X_train[:,i*N+j]=np.float64(imread('mnist/'+dp+'/'+dp+'_'+'%04d.png'%(j+1)).ravel())
        
y_train=np.floor(np.arange(2*N)/N)*2-1

**a)** Consider the equation (8.30) in the lecture notes. Implement a function `solvedualsvm(H,y)` that returns the solution `lambda_star` of the dual SVM problem by means of CVXOPT.

In [2]:
def solvedualsvm(H,y):
    y=y.squeeze()
    n=len(y)
    G=-np.eye(n).astype('double')
    A=y.reshape(1,n).astype('double')
    h=np.zeros((n,)).astype('double')
    b=np.zeros((1,)).astype('double')
    P=H.astype('double')
    q=-np.ones((n,)).astype('double')
    lambda_star=solvers.qp(matrix(P),matrix(q),matrix(G),matrix(h),matrix(A),matrix(b))
    return lambda_star['x']

Test your function with the training data
		\begin{equation*}
		\begin{split}
		\mathbf{x}_1=\begin{bmatrix}-1\\-1\end{bmatrix},y_1=-1,&\ \mathbf{x}_2=\begin{bmatrix}-2\\-2\end{bmatrix},y_2=-1,\\
		\mathbf{x}_3=\begin{bmatrix}1\\1\end{bmatrix},y_3=1,&\ \mathbf{x}_4=\begin{bmatrix}2\\2\end{bmatrix},y_4=1,\
		\end{split}.
		\end{equation*}
		Verify that the KKT conditions with respect to the support vectors are in line with what you expect.

In [3]:
X_toy=np.array([[-1,-2,1,2],[-1,-2,1,2]])
y_toy=np.array([-1,-1,1,1])
H_toy=np.dot(np.dot(np.dot(np.diag(y_toy),X_toy.T),X_toy),np.diag(y_toy))
lambda_star=solvedualsvm(H_toy,y_toy)
print(lambda_star)

     pcost       dcost       gap    pres   dres
 0: -4.8980e-01 -8.9796e-01  6e+00  2e+00  1e+00
 1: -1.8751e-01 -5.7797e-01  4e-01  2e-02  1e-02
 2: -2.4373e-01 -2.8494e-01  4e-02  1e-16  2e-16
 3: -2.4987e-01 -2.5034e-01  5e-04  8e-17  4e-16
 4: -2.5000e-01 -2.5000e-01  5e-06  6e-17  3e-16
 5: -2.5000e-01 -2.5000e-01  5e-08  3e-17  4e-16
Optimal solution found.
[ 2.50e-01]
[ 6.52e-09]
[ 2.50e-01]
[ 6.52e-09]



Only the KKT coefficients that belong to the support vectors are significantly larger than 0.

**b)** Write the function `simplesvm` which expects a training data matrix `X_train`, a training label vector `y_train` and a test data matrix `X_train` as its input. As a result, it returns the estimated test label vector `y_test`. To this end, employ `solvedualsvm` from the last lab course. Note that (8.29) in the lecture notes is overdetermined. You can exploit this to get a more robust estimation of $b$. Test your implementation with another 800 images from the MNIST data set.

In [4]:
def simplesvm(X_train,y_train, X_test=None):
    y_train=y_train.ravel()
    H=np.dot(X_train.T,X_train)*np.expand_dims(y_train,0)*np.expand_dims(y_train,1)
    lambda_star=np.array(solvedualsvm(H,y_train)).ravel()
    w=np.dot(X_train,lambda_star*y_train)
    lambda_sqrd=lambda_star**2
    b=np.dot(lambda_sqrd,np.dot(X_train.T,w)-y_train)/np.sum(lambda_sqrd)
    ret=np.sign(np.dot(X_test.T,w)-b)
    if X_test is None:
        ret=[w,b]
    return ret

N_test=400
digit_prefix=['d1','d2']
X_test=np.zeros((784,2*N_test))
for i,dp in enumerate(digit_prefix):
    for j in range(N_test):
        X_test[:,i*N_test+j]=np.float64(imread('mnist/'+dp+'/'+dp+'_'+'%04d.png'%(j+N+1)).ravel())
y_test=np.floor(np.arange(2*N_test)/N_test)*2-1
        
print('Success rate:', np.sum((simplesvm(X_train, y_train, X_test)*y_test+1))/(4*N_test))

     pcost       dcost       gap    pres   dres
 0: -3.5017e+01 -7.5935e+01  3e+03  5e+01  2e+00
 1: -1.3819e+01 -4.1958e+00  5e+02  6e+00  3e-01
 2: -9.3107e-01 -1.8784e-01  3e+01  4e-01  2e-02
 3: -4.2808e-02 -3.1534e-02  2e+00  3e-02  1e-03
 4: -8.5850e-03 -9.2431e-03  4e-01  5e-03  2e-04
 5: -1.8760e-03 -3.0697e-03  9e-02  9e-04  4e-05
 6: -4.7896e-04 -9.1748e-04  1e-02  1e-04  6e-06
 7: -1.8519e-04 -3.2212e-04  4e-03  3e-05  1e-06
 8: -7.1333e-05 -1.5382e-04  9e-04  7e-06  3e-07
 9: -3.1185e-05 -1.0009e-04  3e-04  2e-06  8e-08
10: -2.4428e-05 -6.4232e-05  6e-05  2e-07  1e-08
11: -3.2614e-05 -5.0103e-05  2e-05  4e-08  2e-09
12: -3.9217e-05 -4.6799e-05  8e-06  2e-09  9e-11
13: -4.2295e-05 -4.5466e-05  3e-06  1e-20  4e-14
14: -4.4424e-05 -4.4881e-05  5e-07  6e-20  4e-14
15: -4.4787e-05 -4.4806e-05  2e-08  6e-20  4e-14
Optimal solution found.
Success rate: 0.97375
